In [ ]:
#cloning indic nlp for hindi text tokenization, steps followed as a given in the tutorial

!pip install indic-nlp-library

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

In [ ]:
!pip install Morfessor

In [ ]:
from pathlib import Path
INDIC_NLP_RESOURCES=str(Path("./indic_nlp_resources").resolve())

In [ ]:
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
#importing required moules, no ml modules imported other than the mentikned one 
#also imported some basic utlity modules 


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import csv
import numpy as np
import math

from itertools import chain
import json

import random
import time
import sys

from collections import Counter
from torch.utils.data import Dataset, DataLoader

import nltk
nltk.download('punkt')
from indicnlp.tokenize import indic_tokenize 

from nltk.tokenize.treebank import TreebankWordDetokenizer
en_detok = TreebankWordDetokenizer()



In [ ]:
#readin the data from data.csv file and seperating test and traing data seperately in 4 different files
##also the lists are used to make data_class object for trainging and test time



src_data = []
tgt_data = []
with open('../input/hi2en-translation/train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1 #ignoring the first line as it do not contains relevant data
        else:
            src_data.append(row[1])
            tgt_data.append(row[2])
            # print(f'\t{row[0]}\t{row[1]}\t{row[2]}')
            line_count += 1
    print(f'Processed {line_count} lines.')

origninal_stdout = sys.stdout

#splitting the data into a split of 8 is to 2 and printing to relevant files
#seperated data for any future use if needed
#currently only using training data for voacb genration used later for embedings
leng = len(src_data)
leng = math.ceil(leng*0.8)

index_array = list(range(len(src_data)))
# np.random.shuffle(index_array)

index_train = index_array[:leng]
index_test = index_array[leng:]

with open('hi_train.txt', 'w') as f:
    sys.stdout = f
    for i in index_train:
        txt = src_data[i]
        print(f'{txt}')
    sys.stdout = origninal_stdout

with open('eng_train.txt', 'w') as f:
    sys.stdout = f
    for i in index_train:
        txt = tgt_data[i]
        print(f'{txt}')
    sys.stdout = origninal_stdout

with open('hi_test.txt', 'w') as f:
    sys.stdout = f
    for i in index_test:
        txt = src_data[i]
        print(f'{txt}')
    sys.stdout = origninal_stdout

with open('eng_test.txt', 'w') as f:
    sys.stdout = f
    for i in index_test:
        txt = tgt_data[i]
        print(f'{txt}')
    sys.stdout = origninal_stdout



In [ ]:
#utility fintion to pad sents to equal length
#input list[list[str]] output :list[list[str]] but the output has all the inner list of same length
#pading token used, ignored during  traing

def pad_sents(sents, pad_token):
    sents_padded = []

    
    slen = []
    for sent in sents:
        slen.append(len(sent))
    m_len = max(slen)
    # print(f'{m_len}\n')
    for sent in sents:
        # print(sent)
        temp = sent
        # print(temp)
        for i in range(m_len-len(sent)):
            temp.append(pad_token)
        sents_padded.append(temp)



    ### END YOUR CODE

    return sents_padded


In [ ]:
##this function read from the files created in the above step and return a tokenise list of list of str

def read_corpus(file_path, source):
    if source == 'src':
        data = []
        for line in open(file_path):
            sent = indic_tokenize.trivial_tokenize(line)
            data.append(sent)
        return data
    else:
        data = []
        for line in open(file_path):
            sent = nltk.word_tokenize(line)
            # sent = ['<s>'] + sent + ['</s>']
            data.append(sent)
        return data

In [ ]:
#this class is of vocab entry, we have two source and target
##most of the function are of the sake of ease of use and dont serve any additional benefits

class VocabEntry(object):
    def __init__(self, word2id=None):
        if word2id:
            self.word2id = word2id
        else:
            #some predefined words that are added manually
            #if a dict is not already defined, define it and insert the words
            self.word2id = dict()
            self.word2id['<pad>'] = 0   # Pad Token
            self.word2id['<s>'] = 1 # Start Token
            self.word2id['</s>'] = 2    # End Token
            self.word2id['<unk>'] = 3   # Unknown Token
        self.unk_id = self.word2id['<unk>']  #we are storing the token of unknown words as when our data will not be able to recognise a word it will bw assigned to unk
        self.id2word = {v: k for k, v in self.word2id.items()}  #returns the index of the word in the dictionary

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)  #if word not present return unk else index of the world

    def __contains__(self, word):
        return word in self.word2id  

    def __setitem__(self, key, value):
        raise ValueError('vocabulary is readonly')  #dict value can't be changed after initialisation

    def __len__(self):
        return len(self.word2id)  #len of dict

    def __repr__(self):
        return 'Vocabulary[size=%d]' % len(self)  #repr of dict when it is preinted

    #add a new world in a dictionary, only if its not already present
    def add(self, word):
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]

    #convert a list of list of str or list of str into list of list on indices or list of indices resp
    def words2indices(self, sents):
        if type(sents[0]) == list:
            return [[self[w] for w in s] for s in sents]
        else:
            return [self[w] for w in sents]
    #convert list of index into words
    def indices2words(self, word_ids):
        return [self.id2word[w_id] for w_id in word_ids]

    ##given a list of list of str convert it into a torch tensor with sents being padded and of the shape (mx_sec_length, batch_size, embedding_size)
    def to_input_tensor(self, sents) -> torch.Tensor:
        word_ids = self.words2indices(sents)
        sents_t = pad_sents(word_ids, self['<pad>'])
        sents_var = torch.tensor(sents_t, dtype=torch.long)
        return torch.t(sents_var)

    @staticmethod
    def from_corpus(corpus, size, freq_cutoff=2):
        vocab_entry = VocabEntry()
        word_freq = Counter(chain(*corpus))
        valid_words = [w for w, v in word_freq.items() if v >= freq_cutoff]
        print('number of word types: {}, number of word types w/ frequency >= {}: {}'
              .format(len(word_freq), freq_cutoff, len(valid_words)))
        top_k_words = sorted(valid_words, key=lambda w: word_freq[w], reverse=True)[:size]
        for word in top_k_words:
            vocab_entry.add(word)
        return vocab_entry


In [ ]:
class Vocab(object):
    def __init__(self, src_vocab: VocabEntry, tgt_vocab: VocabEntry):
        self.src = src_vocab
        self.tgt = tgt_vocab

    @staticmethod
    def build(src_sents, tgt_sents, vocab_size, freq_cutoff) -> 'Vocab':
        assert len(src_sents) == len(tgt_sents)

        print('initialize source vocabulary ..')
        src = VocabEntry.from_corpus(src_sents, vocab_size, freq_cutoff)

        print('initialize target vocabulary ..')
        tgt = VocabEntry.from_corpus(tgt_sents, vocab_size, freq_cutoff)

        return Vocab(src, tgt)

    def __repr__(self):
        return 'Vocab(source %d words, target %d words)' % (len(self.src), len(self.tgt))



In [ ]:
src_sents = read_corpus('hi_train.txt', source='src')
tgt_sents = read_corpus('eng_train.txt', source='tgt')


In [ ]:
 vocab_train = Vocab.build(src_sents, tgt_sents, 30000, 2)

In [ ]:
# vocab.save('vocab.json')

In [ ]:
class LangDataset(Dataset):
    def __init__(self, tot):
        # Initialize data, download, etc.
        # read with numpy or pandastokenize(row[1]))
        leng = len(src_data)
        leng = math.ceil(leng*0.8)
        if (tot=='train'):
          self.src, self.tgt = src_data[:leng], tgt_data[:leng]        # print(f'ads')
        if (tot=='test'):
          self.src, self.tgt = src_data[leng:], tgt_data[leng:]

        self.n_samples = len(self.src)

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.src[index], self.tgt[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [ ]:
# vocab_train = Vocab.load('vocab.json')

print(vocab_train)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim,dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, bidirectional = True)
        self.h_proj = nn.Linear(hid_dim*2, hid_dim)
        self.c_proj = nn.Linear(hid_dim*2, hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)

        hidden = self.h_proj(torch.cat((hidden[0], hidden[1]), 1))
        cell =  self.c_proj(torch.cat((cell[0], cell[1]), 1))

        #output (s, b, 2h) hidden-(1, b, h) cell - (1, b, h)
        return outputs, (hidden.unsqueeze(0), cell.unsqueeze(0))


In [ ]:
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        
        self.com_proj = nn.Linear(hid_dim*3, hid_dim)
        self.dropout = nn.Dropout(0.2)
        
        # self.v = nn.Linear(hid_dim, 1, bias = False)
        
    def forward(self, dec_state, encoder_outputs, encoder_outputs_proj):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden, cell = dec_state
        e_t =torch.bmm(encoder_outputs_proj, hidden.squeeze(0).unsqueeze(2)).squeeze(2)

        alpha_t = F.softmax(e_t)
        a_t = torch.bmm(alpha_t.unsqueeze(1), encoder_outputs).squeeze(1)#(b, 2h)

        U_t = torch.cat((a_t, hidden.squeeze(0)), dim=1) #(b, 3h)
        V_t = self.com_proj(U_t)
        O_t = self.dropout(torch.tanh(V_t))

        return O_t





In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, enc_dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.hid_dim = hid_dim
        

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim)
        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.att_proj = nn.Linear(2*hid_dim, hid_dim)
        self.dropout1 = nn.Dropout(enc_dropout)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.dropout1(self.embedding(input))

        _, dec_state = self.rnn(embedded, (hidden, cell))
        
        # dec_state = (hidden.squeeze(0), cell.squeeze(0))
        (hidden, cell) = dec_state
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #enc_ot = (b, s, 2h)
        encoder_outputs_proj = self.att_proj(encoder_outputs)
        #enc_out = (b,s, h)

        prediction = self.attention(dec_state, encoder_outputs, encoder_outputs_proj)
        prediction_proj = self.fc_out(prediction)


        
        return prediction_proj, (hidden, cell)


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, "Hid_dim should be same"
        
    
    def forward(self,src, trg, teacher_forcing_ratio = 0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, (hidden, cell) = self.encoder(src)

        input = trg[0,:]

        for t in range(1, trg_len):
            output, (hidden, cell) = self.decoder(input, hidden, cell, encoder_outputs)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)

            input = trg[t] if teacher_force else top1
        return outputs


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        if i%10 == 0:
            print(f'{i}iteration')
        # print(f'{i}iteration')
        src, trg = batch
        src = list(src) #convert to list of sentences
        src = [["<s>"] +indic_tokenize.trivial_tokenize(sent) + ["</s>"] for sent in src]
        src = vocab_train.src.to_input_tensor(src).to(device)

        trg = list(trg)
        trg = [["<s>"] + nltk.word_tokenize(sent) + ["</s>"] for sent in trg]
        trg = vocab_train.tgt.to_input_tensor(trg).to(device)
        # print(trg.shape)

        optimizer.zero_grad()

        output = model(src, trg)

        # print(output.shape)
        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        # print(output.shape)
        trg = trg.contiguous()[1:].view(-1)
        # print(trg.shape)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss/len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            if (i%20 == 0):
                print(f'{i}iteration')
            # print(f'{i}iteration')
            src, trg = batch
            src = list(src) #convert to list of sentences
            src = [["<s>"] + indic_tokenize.trivial_tokenize(sent) + ["</s>"] for sent in src]
            src = vocab_train.src.to_input_tensor(src).to(device)

            trg = list(trg)
            trg = [["<s>"] + nltk.word_tokenize(sent) + ["</s>"] for sent in trg]
            trg = vocab_train.tgt.to_input_tensor(trg).to(device)
            # print(trg.shape)
            output = model(src, trg, 0)

            a = output.argmax(2)
            # print(a.shape)
            a = torch.transpose(a, 0, 1)
            # print(a.shape)
            a = a.tolist()
            for sent in a:
                o_sent = vocab_train.tgt.indices2words(sent)

            # print(a)

            # print(output)


            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)

            trg = trg.contiguous()[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

        return epoch_loss/len(iterator)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = LangDataset('train')

test_dataset = LangDataset('test')

In [ ]:
BATCH_SIZE = 16

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size= BATCH_SIZE)
test_loader = DataLoader(dataset= test_dataset, batch_size= BATCH_SIZE)


In [ ]:
INPUT_DIM = len(vocab_train.src)
OUTPUT_DIM = len(vocab_train.tgt)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5


In [ ]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
attention = Attention(HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT, attention)

model = Seq2Seq(enc, dec, device).to(device)
print(f'The model has {count_parameters(model):,} trainable parameters') 
TRG_PAD_IDX = vocab_train.tgt['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index= TRG_PAD_IDX)

optimizer = optim.Adam(model.parameters())

In [ ]:
N_EPOCHS = 8
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
#         torch.save(model.state_dict(), './drive/MyDrive/nmt/model_att1.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')



In [ ]:
enc1 = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
attention = Attention(HID_DIM)
dec1 = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT, attention)

model1 = Seq2Seq(enc1,dec1, device).to(device)

model1.load_state_dict(torch.load('model.pt', map_location=device))

# test_dataset = LangDataset('test.csv')

# test_loader = DataLoader(dataset= test_dataset, batch_size= 8)
# TRG_PAD_IDX = vocab_train.tgt['<pad>']
# criterion = nn.CrossEntropyLoss(ignore_index= TRG_PAD_IDX)
# test_loss = evaluate(model, test_loader, criterion)
# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')
src_data = []
with open('../input/eval-nmt-trans/testhindistatements.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        # print(row)
        if(line_count == 0):
            line_count += 1
            continue
        src_data.append(row[2])
        line_count += 1

    print(f'Processed {line_count} lines.')

print(len(src_data))
l = 0
f = open('answer.txt', 'a', encoding= "utf8")
for sent in src_data:
    sent = ["<s>"] + indic_tokenize.trivial_tokenize(sent) + ["</s>"]
    sent = vocab_train.src.words2indices(sent)
    sent = torch.tensor(sent, dtype=torch.long).to(model1.device)
    sent = sent.unsqueeze(1)
    # print(sent.shape)
    # sent = vocab_train.src.to_input_tensor(sent).to(device)
    # print(sent)
    batch_size = 1
    trg_len = 30
    trg_vocab_size = model1.decoder.output_dim

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(model1.device)

    encoder_outputs, (hidden, cell) = model1.encoder(sent)

    input = torch.tensor([1]).to(model1.device)

    for t in range(trg_len):
        output, (hidden, cell) = model1.decoder(input, hidden, cell, encoder_outputs)

        outputs[t] = output

        top1 = output.argmax(1)

        input = top1
    
    word_vec = outputs.argmax(2) #teg*b*output_dim
    word_vec = torch.transpose(word_vec, 0,1)
    # print(word_vec.shape)
    word_vec = torch.reshape(word_vec, (-1,))
    # print(word_vec.shape)
    # word_vec = list(word_vec)
    # print(len(word_vec))
    final_sent = []
    for i in range(29):
        # print(word_vec[i])
        if ( word_vec[i].item() == 2):
            break

                
        wrd = vocab_train.tgt.id2word[word_vec[i].item()]
        final_sent.append(wrd)
        
    # f = open('final_out.txt', 'a', encoding= "utf8")
    # f.write()
    # print()
    if (l < len(src_data)-1):
        f.write(en_detok.detokenize(final_sent) + "\n")

    else:
        f.write(en_detok.detokenize(final_sent) )
    l += 1
f.close()


In [ ]:
# ! nvidia-smi